<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/transform/simple">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/transform/simple.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/transform/simple.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
</table></div>

##### Copyright &copy; 2020 Google Inc.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TensorFlow Transform 预处理数据

***TensorFlow Extended (TFX) 的特征工程组件***

此示例 Colab 笔记本提供了一个非常简单的示例，说明了如何使用 <a target="_blank" href="https://www.tensorflow.org/tfx/transform/">TensorFlow Transform</a> (<code>tf.Transform</code>) 预处理数据，此示例使用完全相同的代码训练模型和在生产环境中应用推断。

TensorFlow Transform 是一个用于预处理 TensorFlow 输入数据的库，包括创建需要在训练数据集上进行完整传递的特征。利用 TensorFlow Transform，您可以：

- 使用平均值和标准差归一化输入值
- 通过在所有输入值上生成词汇将字符串转换为整数
- 根据观测到的数据分布，通过分配给桶将浮点数转换为整数

TensorFlow 内置了对在单个样本或一批样本上进行操作的支持。`tf.Transform` 扩展了这些功能，支持在整个训练数据集上进行完整传递。

`tf.Transform` 的输出将导出为可用于训练和应用的 TensorFlow 计算图。将同一个计算图用于训练和应用可以避免偏差，因为会在两个阶段应用相同的转换。

### 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保我们在 Colab 中运行。当然，可以单独升级本地系统。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Install TensorFlow Transform

**Note: In Google Colab, because of package updates, the first time you run this cell you may need to restart the runtime (Runtime > Restart runtime ...).**

In [ ]:
!pip install tensorflow==2.2.0

## Did you restart the runtime?

If you are using Google Colab, the first time that you run the cell above, you must restart the runtime (Runtime > Restart runtime ...). This is because of the way that Colab loads packages.

## Imports

In [ ]:
import pprint
import tempfile

import tensorflow as tf
import tensorflow_transform as tft

import tensorflow_transform.beam as tft_beam
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

## 数据：创建一些虚拟数据

我们将为此简单示例创建一些简单的虚拟数据：

- `raw_data` 是我们将要预处理的初始原始数据
- `raw_data_metadata` 包含告知我们 `raw_data` 中每个列的类型的架构。在本例中，它非常简单。

In [ ]:
raw_data = [
      {'x': 1, 'y': 1, 's': 'hello'},
      {'x': 2, 'y': 2, 's': 'world'},
      {'x': 3, 'y': 3, 's': 'hello'}
  ]

raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        'y': tf.io.FixedLenFeature([], tf.float32),
        'x': tf.io.FixedLenFeature([], tf.float32),
        's': tf.io.FixedLenFeature([], tf.string),
    }))

## Transform：创建一个预处理函数

<em>预处理函数</em>是 tf.Transform 最重要的概念。预处理函数是真正发生数据集转换的地方。它接受并返回一个张量字典，其中张量是指 <a target="_blank" href="https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/SparseTensor"><code>Tensor</code></a> 或 <a><code>SparseTensor</code></a>。有两组主要的 API 调用，它们通常构成预处理函数的核心：

1. **TensorFlow 运算**：接受并返回张量的任何函数，通常是指 TensorFlow 运算。这些函数会将 TensorFlow 运算添加到计算图中，计算图能够以每次一个特征向量的方式转换原始数据。在训练和应用期间，将为每个样本运行这种转换。
2. **Tensorflow Transform Analyzers/Mappers:** Any of the analyzers/mappers provided by tf.Transform. These also accept and return tensors, and typically contain a combination of Tensorflow ops and Beam computation, but unlike TensorFlow ops they only run in the Beam pipeline during analysis requiring a full pass over the entire training dataset. The Beam computation runs only once, during training, and typically make a full pass over the entire training dataset. They create tensor constants, which are added to your graph. For example, tft.min computes the minimum of a tensor over the training dataset while tft.scale_by_min_max first computes the min and max of a tensor over the training dataset and then scales the tensor to be within a user-specified range, [output_min, output_max]. tf.Transform provides a fixed set of such analyzers/mappers, but this will be extended in future versions.

Caution: When you apply your preprocessing function to serving inferences, the constants that were created by analyzers during training do not change.  If your data has trend or seasonality components, plan accordingly.

Note: The `preprocessing_fn` is not directly callable. This means that calling `preprocessing_fn(raw_data)` will not work. Instead, it must be passed to the Transform Beam API as shown in the following cells.

In [ ]:
def preprocessing_fn(inputs):
    """Preprocess input columns into transformed columns."""
    x = inputs['x']
    y = inputs['y']
    s = inputs['s']
    x_centered = x - tft.mean(x)
    y_normalized = tft.scale_to_0_1(y)
    s_integerized = tft.compute_and_apply_vocabulary(s)
    x_centered_times_y_normalized = (x_centered * y_normalized)
    return {
        'x_centered': x_centered,
        'y_normalized': y_normalized,
        's_integerized': s_integerized,
        'x_centered_times_y_normalized': x_centered_times_y_normalized,
    }

## 总结

现在，我们准备转换数据。我们将通过直接运行程序使用 Apache Beam，并提供三个输入：

1. `raw_data` - 我们在上面创建的原始输入数据
2. `raw_data_metadata` - 原始数据的架构
3. `preprocessing_fn` - 我们创建用于进行转换的函数

<aside class="key-term"><b>关键词</b>：<a target="_blank" href="https://beam.apache.org/">Apache Beam</a> 使用<a target="_blank" href="https://beam.apache.org/documentation/programming-guide/#applying-transforms">特殊的语法来定义和调用 transform</a>。例如，在下面的代码行中：</aside>

<code><blockquote>result = pass_this | 'name this step' &gt;&gt; to_this_call</blockquote></code>

方法 <code>to_this_call</code> 正在被调用并传递给名为 <code>pass_this</code> 的对象，<a target="_blank" href="https://stackoverflow.com/questions/50519662/what-does-the-redirection-mean-in-apache-beam-python">在堆栈跟踪中，此运算被称为 <code>name this step</code></a>。调用 <code>to_this_call</code> 的结果将在 <code>result</code> 中返回。您经常会看到流水线的各个阶段像这样链接在一起：

<code><blockquote>result = apache_beam.Pipeline() | 'first step' &gt;&gt; do_this_first() | 'second step' &gt;&gt; do_this_last()</blockquote></code>

由于这是从一个新的流水线开始的，因此您可以按以下方式继续：

<code><blockquote>next_result = result | 'doing more stuff' &gt;&gt; another_function()</blockquote></code>

In [ ]:
def main():
  # Ignore the warnings
  with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    transformed_dataset, transform_fn = (  # pylint: disable=unused-variable
        (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
            preprocessing_fn))

  transformed_data, transformed_metadata = transformed_dataset  # pylint: disable=unused-variable

  print('\nRaw data:\n{}\n'.format(pprint.pformat(raw_data)))
  print('Transformed data:\n{}'.format(pprint.pformat(transformed_data)))

if __name__ == '__main__':
  main()

## 这是正确答案吗？

以前，我们使用 `tf.Transform` 来做到这一点：

```
x_centered = x - tft.mean(x)
y_normalized = tft.scale_to_0_1(y)
s_integerized = tft.compute_and_apply_vocabulary(s)
x_centered_times_y_normalized = (x_centered * y_normalized)
```

####x_centered With input of `[1, 2, 3]` the mean of x is 2, and we subtract it from x to center our x values at 0.  So our result of `[-1.0, 0.0, 1.0]` is correct. ####y_normalized We wanted to scale our y values between 0 and 1.  Our input was `[1, 2, 3]` so our result of `[0.0, 0.5, 1.0]` is correct. ####s_integerized We wanted to map our strings to indexes in a vocabulary, and there were only 2 words in our vocabulary ("hello" and "world").  So with input of `["hello", "world", "hello"]` our result of `[0, 1, 0]` is correct. Since "hello" occurs most frequently in this data, it will be the first entry in the vocabulary. ####x_centered_times_y_normalized We wanted to create a new feature by crossing `x_centered` and `y_normalized` using multiplication.  Note that this multiplies the results, not the original values, and our new result of `[-0.0, 0.0, 1.0]` is correct.